In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Spark Miniproject


Stack Overflow is a collaboratively edited question-and-answer site originally focused on programming topics. Because of the variety of features tracked, including a variety of feedback metrics, it allows for some open-ended analysis of user behavior on the site.

Stack Exchange (the parent organization) provides an anonymized [data dump](https://archive.org/details/stackexchange), and we'll use Spark to perform data manipulation, analysis, and machine learning on this data set. As a side note, there's also an online data explorer which allows you to query the data interactively.

## Accessing the data


The data is available on S3 (s3://dataincubator-course/spark-stack-data). There are three sub-folders, `allUsers`, `allPosts`, and `allVotes`, which contain Gzipped XML with the following format:

```
<row Body="&lt;p&gt;I always validate my web pages, and I recommend you do the same BUT many large company websites DO NOT and cannot validate because the importance of the website looking exactly the same on all systems requires rules to be broken. &lt;/p&gt;&#10;&#10;&lt;p&gt;In general, valid websites help your page look good even on odd configurations (like cell phones) so you should always at least try to make it validate.&lt;/p&gt;&#10;" CommentCount="0" CreationDate="2008-10-12T20:26:29.397" Id="195995" LastActivityDate="2008-10-12T20:26:29.397" OwnerDisplayName="Eric Wendelin" OwnerUserId="25066" ParentId="195973" PostTypeId="2" Score="0" />
```

Data from the much smaller stats.stackexchange.com is available in the same format on S3 (s3://dataincubator-course/spark-stats-data). This site, Cross-Validated, will be used below in some instances to avoid working with the full data set for every question.

The full schema is available as a text file:

In [3]:
!aws s3 cp s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt .

download: s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt to ./stack_exchange_schema.txt


You can either get the data by running the appropriate S3 commands in the terminal, or by running this block for the smaller stats data set:

In [4]:
!mkdir -p spark-stats-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data
!aws s3 sync --exclude '*' --include 'posts*zip' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data

And to get the much larger full data set (be warned, this can take 20 or more minutes, so you may want to run it in the terminal to avoid locking up the notebook):

In [5]:
!mkdir -p spark-stack-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stack-data/ ./spark-stack-data

## Data input and parsing


Some rows are split across multiple lines; these can be discarded. Incorrectly formatted XML can also be ignored. It is enough to simply skip problematic rows, the loss of data will not significantly impact our results on this large data sets.

You will need to handle XML parsing yourself using the `\` selector in Scala or something like `lxml.etree` in Python. *Warning*: The built-in `xml.etree.ElementTree` behaves differently and the results don't correspond perfectly with the Scala equivalent.

To make your code more flexible, it's also recommended to incorporate command-line arguments that specify the location of the input data and where output should be written.

The goal should be to have a parsing function that can be applied to the input data to access any XML element desired. It is suggested to use a class structure so that you can create RDDs of Posts, Votes, Users, etc.

``` scala
// Command line arguments in Scala

object Main {
 def main(args: Array[String]) {
   val inputDir = args(0)
   val outputDir = args(1)
   ...
```

``` python

# Command line arguments using sysv or argparse in Python

if __name__ == '__main__':
    main(ARGS.input_dir, ARGS.output_dir)
```

Dates are parsed by default using the Long data type and Unix time (epoch time). In Java/Scala, a given timestamp represents the number of milliseconds since 1970-01-01T00:00:00Z. Also be wary of integer overflow when dealing with Longs. For example, these two are not equal:

`val year: Long = 365 * 24 * 60 * 60 * 1000`

`val year: Long = 365 * 24 * 60 * 60 * 1000L`

## Questions

## Bad XML


A simple question to test your parsing code. Create an RDD of Post objects where each Post is a valid row of XML from the Cross-Validated (stats.stackexchange.com) `allPosts` data set.

We are going to take several shortcuts to speed up and simplify our computations.  First, your parsing function to only attempt to parse rows that start with `  <row` as these denote actual data entries. This should be done in Spark as the data is being read in from disk, without any pre-Spark processing. 

Return the total number XML rows that started with ` <row` that were subsequently **rejected** during your processing.  Note that the text is unicode, and contains non-ASCII characters.  You may need to re-encode to UTF-8 (depending on your XML parser)

Note that this cleaned data set will be used for all subsequent questions.

*Question*: Can you figure out what filters you need to put in place to avoid throwing parsing errors entirely?

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")

import os, time
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)

In [3]:
posts_xml = sc.textFile(localpath('spark-stats-data/allPosts/*'))

In [4]:
votes_xml = sc.textFile(localpath('spark-stats-data/allVotes/*'))

In [5]:
users_xml = sc.textFile(localpath('spark-stack-data/allUsers/*'))

In [8]:
from lxml import etree

def xmlparser(xmlstring):
    try:
        root = etree.fromstring(xmlstring.encode('utf-8'), etree.XMLParser(encoding='utf-8'))
        return root.attrib
    except:
        return None

In [ ]:
def isRow(line):
    return "<row" in line

In [9]:
class recordPost(object):
    def __init__(self, tags, userID, date, typeID, score, views, answers, favorites, answerID):
        self.tags = tags
        self.userID = userID
        self.date = date
        self.typeID = typeID
        self.score = score
        self.views = views
        self.answers = answers
        self.favorites = favorites
        self.answerID = answerID

    @classmethod
    def parse(cls, line):
        attributes = xmlparser(line)
        tags = attributes.get('Tags')
        userID = attributes.get('OwnerUserId')
        date = attributes.get('CreationDate')
        typeID = attributes.get('PostTypeId')
        score = int(attributes.get('Score', "0"))
        views = int(attributes.get('ViewCount', "0"))
        answers = int(attributes.get('AnswerCount', "0"))
        favorites = int(attributes.get('FavoriteCount', "0"))
        answerID = attributes.get('AcceptedAnswerId')
        return cls(tags, userID, date, typeID, score, views, answers, favorites, answerID)
    
class recordUser(object):
    def __init__(self, ID, date, reputation):
        self.ID = ID
        self.date = date
        self.reputation = reputation

    @classmethod
    def parse(cls, line):
        attributes = xmlparser(line)
        ID = attributes.get('Id')
        date = attributes.get('CreationDate')
        reputation = attributes.get('Reputation')
        return cls(ID, date, reputation)
    
class recordVote(object):
    def __init__(self, typeID, postID):
        self.typeID = typeID
        self.postID = postID

    @classmethod
    def parse(cls, line):
        attributes = xmlparser(line)
        typeID = attributes.get('VoteTypeId')
        postID = attributes.get('PostId')
        return cls(typeID, postID)
    
class Record(object):
    def __init__(self, attributes):
        self.attr = attributes

    @classmethod
    def parse(cls, line):
        attributes = xmlparser(line)
        return cls(attributes)

In [45]:
post_record = posts_xml.filter(lambda x: isRow(x)) \
                .filter(lambda x: xmlparser(x) is not None) \
                .map(recordPost.parse)
    
vote_record = votes_xml.filter(lambda x: isRow(x)) \
                .filter(lambda x: xmlparser(x) is not None) \
                .map(recordPost.parse)
    
user_record = users_xml.filter(lambda x: isRow(x)) \
                .filter(lambda x: xmlparser(x) is not None) \
                .map(recordPost.parse)

In [46]:
badXMLcount = post_xml.filter(lambda x: isRow(x)) \
                    .filter(lambda x: xmlparser(x) is None) \
                    .count()

In [47]:
badXMLcount

781

In [42]:
def bad_xml():
    return badXMLcount

grader.score(question_name='spark__bad_xml', func=bad_xml)

Your score:  1


## Upvote percentage


Each post on Stack Exchange can be upvoted, downvoted, and favorited. One "sanity check" we can do is to look at the ratio of upvotes to downvotes (referred to as `UpMod` and `DownMod` in the schema) as a function of how many times the post has been favorited.

You might hypothesize, for example, that posts with more favorites should have a higher upvote/downvote ratio.

Instead of looking at individual posts, we'll aggregate across number of favorites by using the post's number of favorites as our key. Since we're computing ratios, bundling together all posts with the same number of favorites effectively averages over them.  Calculate the average percentage of upvotes *(upvotes / (upvotes + downvotes))* for the smallest 50 **keys**.

Do the analysis on the smaller Cross-Validated data set.


#### Checkpoints

* Total upvotes: 313,819
* Total downvotes: 13,019
* Mean of first 50 keys (averaging the keys themselves): 24.76

In [54]:
def voteMap(val):
    # 2 = Upvote, 3 = Downvote, 5 = Favorite
    # (Upvote, Downvote, Favorite)
    vote_dict = {'2':(1,0,0), '3':(0,1,0), '5':(0,0,1)}
    return vote_dict.get(val)

In [71]:
vote_bins = ['2','3','5']

def tupleAdd(x, y):
    return (x[0] + y[0], x[1] + y[1])

def thrupleAdd(x, y):
    return (x[0] + y[0], x[1] + y[1], x[2] + y[2])

def last2Key(x):
    return (x[2], (x[0], x[1]))

def calcFraction(x):
    return (float(x[0]) / (x[0] + x[1]))

def calcRatio(x):
    return (float(x[0])/ x[1])

upvote_ratio = vote_record.filter(lambda x: x.typeID in vote_bins) \
                .map(lambda x: (x.postID, vote_map(x.typeID)) \ # (postID, (Upvotes, Downvotes, Favorites))
                .reduceByKey(lambda x, y: thrupleAdd(x, y)) \ 
                .map(lambda x: last2Key(x[1])) \ # (Favorites, (Upvotes, Downvotes))
                .reduceByKey(lambda x, y: tupleAdd(x, y)) \
                .sortByKey() \
                .mapValues(lambda x: calcFraction(x)) \ # (Favorites, Upvote Ratios)
                .take(50) \

In [74]:
def upvote_percentage():
    return upvote_ratio #return [(20, 0.9952153110047847)] * 50
grader.score(question_name='spark__upvote_percentage', func=upvote_percentage)

Your score:  1.0


## Answer percentage


Investigate the correlation between a user's reputation and the kind of posts they make. For the 99 users with the highest reputation, single out posts which are either questions or answers and look at the percentage of these posts that are answers: *(answers / (answers + questions))*. 

Return a tuple of their **user ID** and this fraction.

You should also return (-1, fraction) to represent the case where you average over all users (so you will return 100 entries total).

Again, you only need to run this on the statistics overflow set.


#### Checkpoints

* Total questions: 52,060
* Total answers: 55,304
* Top 99 users' average reputation: 11893.464646464647

In [318]:
possible_ids = ["1","2"]
# 1 = Question, 2 = Answer: 
# (Answer, Question)
ptype_map = {"2":(1,0), "1":(0,1)}

ptype_BV = sc.broadcast(ptype_map)

In [317]:
users_reputations = user_record.map(lambda x: (x.ID, x.reputation)) \
                    .filter(lambda x: x[0] and x[1])

In [328]:
users_answers_ratios = post_record.filter(lambda x: x.userID and x.typeID in possible_ids) \
                        .map(lambda x:(x.userID, ptype_BV.value[x.typeID])) \ # (userID, (Answers, Questiosn))
                        .reduceByKey(lambda x, y: tupleAdd(x, y))) \
                        .mapValues(lambda x: calcFraction(x))) \ # (userID, Answer Ratio)
            
users_ratio_reps = users_answers_ratios.join(users_reputations) \ # (userID, (Answer Ratio, User Reputation))
                        .map(lambda (userID, (ratio, rep)): (int(rep), (int(userID),ratio))) \ # (Reputation, (userID, Answer Ratio))

In [329]:
def values2KeyVal(x):
    return (x[1][0],x[1][1])

top99_ratio = users_ratios_reps.sortByKey(ascending=False) \ #(Reputation, (userID, Answer Ratio))
                .map(lambda x: values2KeyVal(x)) \ #(userID, Answer Ratio)
                .take(99)
        
avg_ratio = users_ratios_reps.map(lambda x: x[1][1]).mean()
answer_ratio_full = top99 + [(-1, avg)]

In [330]:
def answer_percentage():
    return answer_ratio_full #return [(7071, 0.9107142857142857)] * 100

grader.score(question_name='spark__answer_percentage', func=answer_percentage)

Your score:  0.99


## Post counts


If we use the total number of posts made on the site as a metric for tenure, we can look at the differences between "younger" and "older" users. You can imagine there might be many interesting features - for now just return the top 100 post counts among all users (of all types of posts) and the average reputation for every user who has that count.

In other words, aggregate the cases where multiple users have the same post count.


#### Checkpoints

* Mean of top 100 post counts: 281.51

In [159]:
users_pcount = post_record.filter(lambda x: x.userID is not None) \
                .map(lambda x: (x.userID, 1)) \ # (userID, 1)
                .reduceByKey(lambda x, y: x + y) \ # (userID, post count)
    
post_count_reps = users_pcount.join(users_reputations) \
                .map(lambda (userID, (count, rep)): (int(count), (int(rep), 1))) \ # (post count, rep, pcount count)
                .reduceByKey(lambda x, y: tupleAdd(x, y)) \
                .mapValues(lambda x: calcRatio(x)) \ # (post count, avg reputation)
                .sortByKey(ascending=False) \
                .take(100)

In [163]:
def post_counts():
    return post_count_reps     #return [(118, 3736.5)] * 100

grader.score(question_name='spark__post_counts', func=post_counts)

Your score:  1.0


## Quick answers


How long do you have to wait to get your question answered? Look at the set of ACCEPTED answers which are posted less than three hours after question creation. What is the average number of these "quick answers" as a function of the hour of day the question was asked? You should normalize by how many total accepted answers are garnered by questions posted in a given hour, just like we're counting how many quick accepted answers are garnered by questions posted in a given hour, eg. (quick accepted answers when question hour is 15 / total accepted answers when question hour is 15).

Return a list, whose *i*th element correspond to *i*th hour (e.g. 0 -> midnight, 1 -> 1:00, etc.)

*Note*: When using the `SimpleDateFormat` class in Scala, it's important to account for your machine's local time zone. Our policy will be to use GMT: `hourFormat.setTimeZone(TimeZone.getTimeZone("GMT"))`

*Consider*: What biases are present in our result that we don't account for? How should we handle this?


#### Checkpoints

* Total quick accepted answers: 8,468
* Total accepted answers: 17,096

In [6]:
import datetime
from dateutil import parser

In [168]:
questions_date = post_record.filter(lambda x: x.answerId and x.date and x.typeID == "1") \
                    .map(lambda x:(x.answerID, parser.parse(x.date))) \

In [170]:
accepted_date = post_record.filter(lambda x: x.ID and x.date and x.typeID == "2") \
                    .map(lambda x:(x.ID, parser.parse(x.date)))

In [182]:
three_hours = 3*60*60

def isLessThanTime(start, end, time):
    return int((end - start).total_seconds() < time)

quick_ans = questions_date.join(accepted_date) \
            .map(lambda (_, (q_date, ans_date)): (q_date.hour, (isLessThanTime(q_date,ans_date,three_hours), 1))) \
            .reduceByKey(lambda x, y: tupleAdd(x, y)) \
            .sortByKey() \
            .map(lambda x: calcRatio(x[1]))  \
            .collect()

In [184]:
def quick_answers():
    return quick_ans     #return [0.] * 24

grader.score(question_name='spark__quick_answers', func=quick_answers)

Your score:  1.0


## Identify veterans


It can be interesting to think about what factors influence a user to remain active on the site over a long period of time. In order not to bias the results towards older users, we'll define a time window between 100 and 150 days after account creation. If the user has made a post in this time, we'll consider them active and well on their way to being veterans of the site; if not, they are inactive and were likely brief users.

*Consider*: What other parameterizations of "activity" could we use, and how would they differ in terms of splitting our user base?

*Consider*: What other biases are still not dealt with, after using the above approach?

Let's see if there are differences between the first ever question posts of "veterans" vs. "brief users". For each group separately, average the score, views, number of answers, and number of favorites of the users' **first question**.

*Consider*: What story could you tell from these numbers? How do the numbers support it?


#### Checkpoints

* Total brief users: 24,864
* Total veteran users: 2,027

In [247]:
all_post_stats = post_record.filter(lambda x: x.userID and x.date) \
                .map(lambda x: (x.userID, \
                (parser.parse(x.date), x.score, x.views, x.answers, x.favorites))) \

In [226]:
seconds_100d  = 100*24*60*60
seconds_150d = 150*24*60*60

def isBetweenTimes(start, end, t_short, t_long):
    dt = (end - start).total_seconds()
    return int(dt <= t_long and t_short <= dt)

vet_users_bool = user_record.map(lambda x: (x.ID, parser.parse(x.date))) \
    .filter(lambda x: x[0] and x[1]) \ # (userID, userDate)
    .join(all_post_data) \ # (userID, (userDate, postDate, score, views, answers, favorites))
    .mapValues(lambda (userDate, postInfo): isBetweenTimes(userDate, postInfo[0], seconds_100d, seconds_150d)) # (userID, post date status)
    .reduceByKey(lambda x, y: x or y) # (userID, veteran status)

In [240]:
first_q_date = post_record.filter(lambda x: x.userID and x.date and x.postID == "1") \
    .map(lambda x: (x.userID, parser.parse(x.date))) \ # (userID, postDate)
    .reduceByKey(lambda x, y : min(x, y)) \ # (userID, first question date)
    .map(lambda (userID, postdate): ((userID, postdate), None)) # ((userID, first question date), _)

In [252]:
post_stats_transform = all_post_stats.map(
    lambda (userID, stats): ((userID, stats[0]), stats[1:])) \ #((userID, postDate), (score,views,answers,favs))

first_q_stats = first_q_date.join(post_stats_transform) \ #((userID, first question Date), (None, stats))
    .map(lambda ((userID, first_q_date), first_q_stats): (userID, first_q_stats[1])) # (userID, stats)

In [278]:
def addQuintuple(x, y):
    return (x[0] + y[0], x[1] + [1], x[2] + y[2], x[3] + y[3], x[4] + y[4])

def computeAvgs(x):
    avgs = (float(x[0]) / x[4], float(x[1]) / x[4], float(x[2]) / x[4], float(x[3]) / x[4])
    return avgs

avg_agg_stats = vet_users_bool.join(first_q_stats) \ # (userID, (vet stats, first question stats))
    .map(lambda (_, (vet_status, first_q_stats)): (vet_status, (first_q_stats, 1))) \ # (vet status, (q stats, 1))
    .mapValues(lambda ((score, views, ans, favs), counter): (score, views, ans, favs, counter)) \
    .reduceByKey(lambda x, y: (addQuintuple(x, y))) \ # (vet status, (score, views, ans, favs, total count))
    .mapValues(lambda x: computeAvgs (x)) \ # (veteran status, (avg stats))
    .collect()

In [303]:
keys = ["vet_score","vet_views","vet_answers","vet_favorites",
        "brief_score","brief_views","brief_answers","brief_favorites"
        ]

In [304]:
values = [el for k, v in sorted(avg_agg_stats, reverse = True) for el in v]
vet_ans_dict = dict(zip(keys, values))
vet_ans_dict

{'brief_answers': 0.9705827067669173,
 'brief_favorites': 0.5757988721804511,
 'brief_score': 2.100892857142857,
 'brief_views': 553.4432330827068,
 'vet_answers': 1.2974161627267728,
 'vet_favorites': 1.3001649257833974,
 'vet_score': 3.54370533260033,
 'vet_views': 925.8889499725124}

In [305]:
def identify_veterans():
    #return {"vet_score": 0,
    #        "vet_views": 0,
    #        "vet_answers": 0,
    #        "vet_favorites": 0,
    #        "brief_score": 0,
    #        "brief_views": 0,
    #        "brief_answers": 0,
    #        "brief_favorites": 0
    #       }
    return v_ans_dict
grader.score(question_name='spark__identify_veterans', func=identify_veterans)

Your score:  1.0


## Word2vec


Word2Vec is an alternative approach for vectorizing text data. The vectorized representations of words in the vocabulary tend to be useful for predicting other words in the document, hence the famous example "vector('king') - vector('man') + vector('woman') ~= vector('queen')".

Let's see how good a Word2Vec model we can train using the tags of each Stack Exchange post as documents (this uses the full data set). Use the implementation of Word2Vec from Spark ML (this will require using DataFrames) to return a list of the top 25 closest synonyms to "ggplot2" and their similarity score in tuple format ("string", number).


#### Parameters


The dimensionality of the vector space should be 100. The random seed should be 42L.


#### Checkpoints

* Mean of the top 25 cosine similarities: 0.7785175901170094

In [9]:
from pyspark.ml.feature import Word2Vec
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

tags_df = sc.textFile(localpath('spark-stack-data/allPosts/*')) \
    .filter(lambda x: isRow(x)) \
    .map(Record.parse) \
    .filter(lambda x: x.attr is not None and x.attr.get('Tags')) \
    .map(lambda x: (x.attr['Tags'].strip('<>').split('><'), 1)) \
    .toDF(['tag', 'score'])

In [351]:
VECTOR_SIZE = 100
SEED = 42L
TOP_N = 25

w2v = Word2Vec(vectorSize = VECTOR_SIZE, seed = SEED, inputCol= "tag", outputCol= "vectors")
model = w2v.fit(tags_df)

In [356]:
synonyms = model.findSynonyms('ggplot2', TOP_N).rdd.collect
syn_list = [(k,v) for (k,v) in synonyms]

In [357]:
def word2vec():
    return syn_list #return [("data.frame", 0.7900882217638416)] * 25

grader.score(question_name='spark__word2vec', func=word2vec)

Your score:  0.96


## Classification


We'd like to see if we can predict the tags of a question from its body text. Technically this is a multi-label classification problem, but to simplify things we'll use a one-vs-all approach where we choose the top k most common tags and train k binary classifiers where the labels indicate the presence or absence of that tag.

Use a logistic regression model as your classifier.

Since we can't reliably save and load models, return a list of 100 tuples ("string", [number, number, number,...]) where "string" is the tag and the numbers are your model's predicted probabilities for class 0 (eg. 0.2 means a prediction that the tag is present) across the test set.

Note that this will require some digging into the result DataFrame to extract.
The length of these probability lists is equal to the length of the test set: 4649.


#### Parameters


* The training and tests sets were downloaded with the stats data at the beginning of the notebook.  You can also get them from S3:
  * s3://dataincubator-course/spark-stats-data/posts_train.zip
  * s3://dataincubator-course/spark-stats-data/posts_test.zip
* Tokenize the body text into words
* number of tags to consider k = 100

In [26]:
from pyspark.ml.feature import Word2Vec
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [10]:
tags_100 = sc.textFile(localpath('spark-stats-data/allPosts/*')) \
    .filter(lambda x: isRow(x)) \
    .map(Record.parse) \
    .filter(lambda x: x.attr is not None and x.attr.get('Tags')) \
    .flatMap(lambda x: (x.attr['Tags'].strip('<>').split('><'))) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[1], x[0])) \
    .sortByKey(ascending = False) \
    .take(100)

In [11]:
tags_100_words = [v for k,v in tags_100]
topwords_BV = sc.broadcast(tags_100_words)

In [61]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [13]:
import mwparserfromhell as mwp

def bodyParser(body):
    try:
        return mwp.parse(body).strip_code().replace('\n',' ')
    except:
        return ''

In [106]:
train = sc.textFile(localpath('posts_train/*')) \
    .filter(lambda x: isRow(x)) \
    .map(Record.parse) \
    .filter(lambda x: x.attr is not None and x.attr.get('Tags') and x.attr.get('Body')) \
    .map(lambda x: (bodyParser(x.attr['Body']), x.attr['Tags'].strip('<>').split('><'))) \
    .filter(lambda x: x[0]) \
    .mapValues(lambda x: [int(word in x) for word in topwords_BV.value])\
    .map(lambda x: [x[0]] + x[1])

In [107]:
test = sc.textFile(localpath('posts_test/*')) \
    .filter(lambda x: isRow(x)) \
    .map(Record.parse) \
    .filter(lambda x: x.attr is not None and x.attr.get('Tags') and x.attr.get('Body')) \
    .map(lambda x: (bodyParser(x.attr['Body']), x.attr['Tags'].strip('<>').split('><'))) \
    .filter(lambda x: x[0]) \
    .mapValues(lambda x: [int(word in x) for word in topwords_BV.value]) \
    .map(lambda x: [x[0]] + x[1])

In [108]:
from pyspark.ml.feature import Word2Vec
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

training = sqlContext.createDataFrame(train, ["text"] + tags_100_words).cache()
testing = sqlContext.createDataFrame(test, ["text"] + tags_100_words)

In [109]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import VectorSlicer

In [110]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

tokens = tokenizer.transform(training)
hashes = hashingTF.transform(tokens)

test_tokens = tokenizer.transform(testing)
test_hashes = hashingTF.transform(test_tokens)

In [112]:
results = []
for i, word in enumerate(tags_100_words):    

    logreg = LogisticRegression(labelCol=word, predictionCol ='pred_' + word, probabilityCol='prob_' + word,
                                maxIter=10, regParam=0.01)
    model = logreg.fit(hashes)
    print(i,word)
    
# Make predictions on test documents
    prediction = model.transform(test_hashes)
    selected = prediction.select("prob_" + word).collect()
    prob_list = [x[0][0] for x in selected]
    results.append((word, prob_list))

(0, 'r')
(1, 'regression')
(2, 'time-series')
(3, 'machine-learning')
(4, 'probability')
(5, 'hypothesis-testing')
(6, 'distributions')
(7, 'self-study')
(8, 'logistic')
(9, 'correlation')
(10, 'classification')
(11, 'statistical-significance')
(12, 'bayesian')
(13, 'anova')
(14, 'normal-distribution')
(15, 'clustering')
(16, 'mathematical-statistics')
(17, 'confidence-interval')
(18, 'data-visualization')
(19, 'multiple-regression')
(20, 'estimation')
(21, 'categorical-data')
(22, 'mixed-model')
(23, 'generalized-linear-model')
(24, 'spss')
(25, 'variance')
(26, 'repeated-measures')
(27, 'sampling')
(28, 't-test')
(29, 'pca')
(30, 'svm')
(31, 'forecasting')
(32, 'multivariate-analysis')
(33, 'chi-squared')
(34, 'cross-validation')
(35, 'maximum-likelihood')
(36, 'data-mining')
(37, 'modeling')
(38, 'neural-networks')
(39, 'data-transformation')
(40, 'predictive-models')
(41, 'matlab')
(42, 'nonparametric')
(43, 'interaction')
(44, 'survival')
(45, 'model-selection')
(46, 'p-value')
(4

In [114]:
def classification():
    return results #return [("repeated-measures", [0.0] * 4649)] * 100

grader.score(question_name='spark__classification', func=classification)

Your score:  0.97


*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*